In [2]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


In [3]:
Toronto_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match="Toronto")
len(Toronto_data)

1

In [4]:
df=Toronto_data[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20 non-null     object
 1   1       20 non-null     object
 2   2       20 non-null     object
 3   3       20 non-null     object
 4   4       20 non-null     object
 5   5       20 non-null     object
 6   6       20 non-null     object
 7   7       20 non-null     object
 8   8       20 non-null     object
dtypes: object(9)
memory usage: 1.5+ KB


In [5]:
df.head

<bound method NDFrame.head of                                                     0  \
0                                     M1ANot assigned   
1                     M1BScarborough(Malvern / Rouge)   
2   M1CScarborough(Rouge Hill / Port Union / Highl...   
3   M1EScarborough(Guildwood / Morningside / West ...   
4                              M1GScarborough(Woburn)   
5                           M1HScarborough(Cedarbrae)   
6                 M1JScarborough(Scarborough Village)   
7   M1KScarborough(Kennedy Park / Ionview / East B...   
8   M1LScarborough(Golden Mile / Clairlea / Oakridge)   
9   M1MScarborough(Cliffside / Cliffcrest / Scarbo...   
10       M1NScarborough(Birch Cliff / Cliffside West)   
11  M1PScarborough(Dorset Park / Wexford Heights /...   
12                 M1RScarborough(Wexford / Maryvale)   
13                          M1SScarborough(Agincourt)   
14  M1TScarborough(Clarks Corners / Tam O'Shanter ...   
15  M1VScarborough(Milliken / Agincourt North / St...   
1

In [6]:
headers=['A','B','C','D','E','F','G','H','I']
df.columns=headers
df.head()

,A,B,C,D,E,F,G,H,I
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [7]:
t=df.stack().reset_index()
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   level_0  180 non-null    int64 
 1   level_1  180 non-null    object
 2   0        180 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.3+ KB


In [8]:
headers=['a','b','c']
t.columns=headers
t.head()

,a,b,c
0,0,A,M1ANot assigned
1,0,B,M2ANot assigned
2,0,C,M3ANorth York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront)


In [9]:
t['PostalCode']=t['c'].str.slice(0,3)
t['D']=t['c'].str.slice(3,)
t.head()

,a,b,c,PostalCode,D
0,0,A,M1ANot assigned,M1A,Not assigned
1,0,B,M2ANot assigned,M2A,Not assigned
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront)


In [10]:
t['D']= t['D'].replace('Not assigned','',)
t.head()

,a,b,c,PostalCode,D
0,0,A,M1ANot assigned,M1A,
1,0,B,M2ANot assigned,M2A,
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront)


In [11]:
t['Borough1']=t['D'].str.replace('(','@(',1)
t

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


,a,b,c,PostalCode,D,Borough1
0,0,A,M1ANot assigned,M1A,,
1,0,B,M2ANot assigned,M2A,,
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods),North York@(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village),North York@(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto@(Regent Park / Harbourfront)
5,0,F,M6ANorth York(Lawrence Manor / Lawrence Heights),M6A,North York(Lawrence Manor / Lawrence Heights),North York@(Lawrence Manor / Lawrence Heights)
6,0,G,M7AQueen's Park(Ontario Provincial Government),M7A,Queen's Park(Ontario Provincial Government),Queen's Park@(Ontario Provincial Government)
7,0,H,M8ANot assigned,M8A,,
8,0,I,M9AEtobicoke(Islington Avenue),M9A,Etobicoke(Islington Avenue),Etobicoke@(Islington Avenue)
9,1,A,M1BScarborough(Malvern / Rouge),M1B,Scarborough(Malvern / Rouge),Scarborough@(Malvern / Rouge)


In [12]:
t[['Borough', 'Neighborhood']]=t['Borough1'].str.split('@', expand=True)
t.head()

,a,b,c,PostalCode,D,Borough1,Borough,Neighborhood
0,0,A,M1ANot assigned,M1A,,,,None
1,0,B,M2ANot assigned,M2A,,,,None
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods),North York@(Parkwoods),North York,(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village),North York@(Victoria Village),North York,(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto@(Regent Park / Harbourfront),Downtown Toronto,(Regent Park / Harbourfront)


In [13]:
t= t.drop(['a','b','c','D','Borough1'],axis=1)
t.dropna(subset=['Neighborhood'], axis=0, inplace=True)
t.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,(Parkwoods)
3,M4A,North York,(Victoria Village)
4,M5A,Downtown Toronto,(Regent Park / Harbourfront)
5,M6A,North York,(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park,(Ontario Provincial Government)


In [14]:
t['Neighborhood']=t['Neighborhood'].str.replace('(','',)
t['Neighborhood']=t['Neighborhood'].str.replace(')','',)
t['Neighborhood']=t['Neighborhood'].str.replace('/',',',)
t.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don MillsNorth
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [22]:
h=t
h.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [21]:
path='https://cocl.us/Geospatial_data'
df2 = pd.read_csv(path)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
ll=df2
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
TD = pd.merge(left=h, right=ll, left_on='PostalCode', right_on='Postal Code')
TD.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [28]:
TD =TD.drop(['Postal Code'],axis=1)
TD.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [30]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="GTA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [31]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(TD['Latitude'], TD['Longitude'], TD['Borough'], TD['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [50]:
CLIENT_ID = 'T4TRSFYJJDDUXPA3CRNFZURCDC25UTBIPBXPQU1ICTVEOR33' # su ID de Foursquare
CLIENT_SECRET = 'PQD2VLPZGXLEUCOB2Z4FLOWJL1ISPJAGZYZ2MNNCFHLIJGUK' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4TRSFYJJDDUXPA3CRNFZURCDC25UTBIPBXPQU1ICTVEOR33
CLIENT_SECRET:PQD2VLPZGXLEUCOB2Z4FLOWJL1ISPJAGZYZ2MNNCFHLIJGUK


## Downtown Toronto

In [42]:
DToronto_data = TD[TD['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [36]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [38]:
map_Dtoronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(DToronto_data['Latitude'], DToronto_data['Longitude'], DToronto_data['Borough'], DToronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Dtoronto)  
    
map_Dtoronto

In [51]:
DToronto_data.loc[0, 'Neighborhood']

'Regent Park , Harbourfront'

In [53]:
neighborhood_latitude = DToronto_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = DToronto_data.loc[0, 'Longitude'] # longitud del barrio

neighborhood_name = DToronto_data.loc[0, 'Neighborhood'] # nombre del barrio

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.6542599, -79.3606359.


In [60]:
search_query = 'Gym'
radius = 500
print(search_query + ' .... OK!')

Gym .... OK!


In [65]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4TRSFYJJDDUXPA3CRNFZURCDC25UTBIPBXPQU1ICTVEOR33&client_secret=PQD2VLPZGXLEUCOB2Z4FLOWJL1ISPJAGZYZ2MNNCFHLIJGUK&ll=43.6542599,-79.3606359&v=20180605&query=Gym&radius=500&limit=100'

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 410,
  'errorType': 'deprecated',
  'errorDetail': 'Usage of the V2 Places API has been deprecated for new Projects. Please see our updated documentation for V3 for more details: https://docs.foursquare.com/reference',
  'requestId': '6266e7f106b4c26d34905943'},
 'response': {}}

In [67]:
# asignar la parte relevante del JSON a la variable venues
venues = results['response']['venues']

# convertir venues en un dataframe
dataframe = json_normalize(venues)
dataframe.head()

KeyError: 'venues'

## North York

In [47]:
nyTORONTO = TD[TD['Borough'] == 'North York'].reset_index(drop=True)
nyTORONTO.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don MillsNorth,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [48]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [49]:
map_nyTORONTO = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(nyTORONTO['Latitude'], nyTORONTO['Longitude'], nyTORONTO['Borough'], nyTORONTO['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyTORONTO)  
    
map_nyTORONTO